In [ ]:
import pandas as pd
import gymnasium as gym
import os
from ale_py import ALEInterface, roms
import time

env = gym.make("ALE/Frogger-v5", render_mode='human')

initial_state = env.reset()
while True:
    env.render()


    # Take a step (0: LEFT, 1: DOWN, 2: RIGHT, 3: UP)
    action = env.action_space.sample()
    new_state, reward, terminated, truncated, info = env.step(action)
    print(action)

    env.render()

time.sleep(5)

env.reset()
env.close()

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


: 

In [ ]:
env.close()